In [2]:
import mlflow

#import torch
#import torch.nn as nn
#from torchinfo import summary
#from torchmetrics import Accuracy
from torch.utils.data import Dataset, DataLoader
from VAE.model import VAE
from VAE.data import Ego4d, DEVICE, BATCH_SIZE, transform1, transform2

device cuda is ready


In [3]:
print('transform initializate sucsess')
train_dataset = Ego4d(img_dir='/home/qwest/data_for_ml/60x60/',
                           transform1=transform1,
                           transform2=transform2)
print("train_dataset init")
train_loader = DataLoader(dataset=train_dataset,
                           batch_size=BATCH_SIZE,
                           shuffle=True,
                           num_workers=14)
print("train_loader init")

transform initializate sucsess
train_dataset init
train_loader init


In [4]:
torch.save(train_loader, "trainloader.pt")

: 

In [ ]:
train_loader = torch.load("VAE/train_loadet.pt")

In [5]:
len(train_loader)

26459

In [6]:
loss_fn = nn.CrossEntropyLoss()

In [5]:
mlflow.set_tracking_uri("http://localhost:8080")

In [1]:
mlflow.set_experiment("/VAE-pytorch")

NameError: name 'mlflow' is not defined

In [8]:
from torch import nn, optim 

In [9]:
lr = 0.001
epochs = 50
latent_dim = 32

model = VAE(latent_dim, batch_size=BATCH_SIZE).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [11]:
def train(epoch):
    #for epoch in range(1, epochs+1):
    x = next(iter(train_loader))
    model.train()
    print(f'Epoch {epoch} start')
    eval_loss = 0
    for batch_idx, data in enumerate(train_loader):
            data = data.to(DEVICE)
            optimizer.zero_grad()
            
            recon_batch, mu, logvar = model(data)
            loss = model.loss_function(recon_batch, data, mu, logvar)
            eval_loss += loss
            loss.backward()
            optimizer.step()

            
    model.eval()
    total_loss = eval_loss/BATCH_SIZE
    mlflow.log_metric("total_loss", f"{total_loss:2f}", step=epoch)
    print(f"Avg loss: {total_loss:2f} \n")
#    recon_img, _, _ = model(x[:1].to(DEVICE))
#    img = recon_img.view(3, 64, 64).detach().cpu().numpy().transpose(1, 2, 0)
        
        #torch.save(train_loader, f'train_loadet_{epochs}.pt')
    #torch.save(model, f'VAE_{epoch}.pt')


In [10]:
#plt.imshow(model(torch.tensor(input)[:1].to(DEVICE))[0][0].view(3, 64, 64).detach().cpu().numpy().transpose(1, 2, 0))

In [12]:
# signature = mlflow.models.infer_signature(
#     input,
#     model(torch.tensor(input)[:1].to(DEVICE))[0][0].view(3, 64, 64).detach().cpu().numpy().transpose(1, 2, 0),
# )

In [223]:
# def evaluation(dataloader, model, loss_fn, epochs):
#     num_batches = len(dataloader)
#     model.eval()
#     eval_loss = 0.0
#     with torch.no_grad():
#         for X in dataloader:
#             Y = model(torch.tensor(X)[:1].to(DEVICE))[0][0].view(3, 64, 64).detach().cpu().numpy()
#             eval_loss += loss_fn(torch.tensor(Y), torch.tensor(X[0])).item()
#     eval_loss /= num_batches
#     mlflow.log_metric("eval_loss", f"{eval_loss:2f}", step=epochs)
#     print(f"Avg loss: {eval_loss:2f} \n")

In [12]:
with mlflow.start_run() as run:
    params = {
        "epochs": epochs,
        "learning_rate": lr,
        "batch_size": BATCH_SIZE,
        #"loss_function": loss_fn.__class__.__name__,
        "metric_function": "BCE",
        "optimizer": "Adam",
    }
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.|
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("model_summary.txt")

    for t in range(1, epochs+1):
        train(t)
        torch.save(model, f'VAE{t}.pt')
        #train(train_dataloader, model, loss_fn, metric_fn, optimizer, epoch=t)
        #evaluation(train_loader, model, loss_fn, epochs=t)

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model, "model")#, signature=signature)
    

Epoch 1 start


: 

In [73]:
# X = model(torch.tensor(input)[:1].to(DEVICE))[0][0].view(3, 64, 64).detach().cpu().numpy()
# Y = x.detach().cpu()

In [81]:
# X.shape

(3, 64, 64)

In [187]:
# loss_fn(torch.tensor(Y), torch.tensor(X)).item()

In [ ]:
with torch.no_grad():
    